# [Module 8] Recommendation API 호출 

# 0. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [4]:
%store -r

# 1. 아이템 테이터 로딩

- 추천 결과의 ITEM_ID 의 상세 내용을 확인하기 위해서 로딩 함.

In [5]:
from src.p_utils import get_item_list_details

items_df = pd.read_csv(items_filename)
items_df.head()

,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,00096972-5f6b-44df-917b-f7d21ae5644c,0.00041,Pink Shirt,apparel,shirt,Swanky dress for women,225.99
1,0016fde3-0910-4cc1-8ef6-90e15f271073,0.00014,Farmed Salmon For Sushi,groceries,seafood,"Flavorful farmed salmon, always sourced sustai...",24.99
2,00225258-dbfb-4103-a573-007386571a49,0.00068,Easter Decorative Egg,seasonal,easter,A must-have for April,16.99
3,003e4953-d6cb-400c-90f6-9b0216b4603e,0.00036,Drought-Resistant Indoor Plant,floral,plant,Drought-resistant indoor plant grown sustainab...,76.99
4,004112e9-dca1-4402-ae6d-74e2b80b8c05,0.00035,Mango Coat,apparel,jacket,Mango coat for men about town,184.99


# 2. Recommendation API 호출

- 두 명의 user_id (에: 1, 100) 에 대해서 추천 결과를 얻음.

### Base 솔루션 vs CVR 가중치 솔루션 추천 : user_id = 1

In [6]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn1,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")    
print("Base Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
Base Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,47636cd1-731c-47d1-8d0b-612575af0a5c,0.00088,Leather Armchair,furniture,chairs,Modish leather armchair for any room,198.99
1,1ddc59b3-7f01-4039-8f07-21a30362ab75,0.00094,Black Office Chair,furniture,chairs,Increase your productivity with this soft blac...,177.99
2,bd86408e-b36c-44b1-ba32-d3f9da0a1af0,0.00005,Taper Candle,homedecor,lighting,This taper candle will enhance your home,21.99
3,79f35543-d013-426e-a2b2-64a03921243f,0.00004,Wooden Coffee Table,furniture,tables,Cool wooden coffee table to enliven your day,165.99
4,c8312844-248a-4a07-96a1-4c02cb1e2471,0.00004,Tan Dining Table,furniture,tables,This definitive tan dining table will delight ...,169.99
5,99b44e8e-65d0-4c55-a7b6-dd1ed0b7cac3,0.00004,Tan Armchair,furniture,chairs,Relax in this comfy tan armchair,214.99
6,1fe3c843-a6b6-432c-8565-456e964543ac,0.00004,White Dresser,furniture,dressers,This unparalleled dresser has lots of drawers ...,184.99
7,52b58d6c-2743-416f-8bbd-da1bbf587182,0.00006,White Armchair,furniture,chairs,Relax in this comfy white armchair,201.99
8,180575fc-ef84-48fc-a83f-067d50141221,0.00019,Dark Gray Office Chair,furniture,chairs,Increase your productivity with this nice dark...,162.99
9,02f7e73f-b6fe-4ffd-bd06-60c6cb35d60f,0.00004,Taper Candle,homedecor,lighting,This taper candle will enhance your home,17.99


In [7]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn2,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")  
print("CVR Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
CVR Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,95677239-43cc-44e4-a97f-4a9c072210ae,0.00083,Gainsboro Armchair,furniture,chairs,Swanky gainsboro armchair for any room,211.99
1,f75f7df9-3ccd-4f27-bcdc-4657e5898740,0.00096,White Dresser,furniture,dressers,This outstanding dresser has lots of drawers t...,230.99
2,dc5bba3f-8d2b-4c81-a482-bbafd457d997,0.00084,Taper Candle,homedecor,lighting,This taper candle will enhance your home,43.99
3,7f4248fb-69ed-46d5-b038-9502b662ad13,0.00082,Tan Accent Table,furniture,tables,Funky tan accent table to enhance your room,181.99
4,1ddc59b3-7f01-4039-8f07-21a30362ab75,0.00094,Black Office Chair,furniture,chairs,Increase your productivity with this soft blac...,177.99
5,124db2fa-17c0-4e94-9844-d1b64a081df5,0.00004,White Dining Chair,furniture,chairs,Outstanding white chair for any dining room,173.99
6,8ac8be2d-9199-4d71-ac9e-0fa9e815b453,0.00082,Wiker Basket,homedecor,decorative,This wiker basket will delight everyone,43.99
7,7719893d-b5d8-4277-ac66-b2fb3394063e,0.00115,White Work Table,furniture,tables,Increase your productivity with this unsurpass...,153.99
8,c6d7f153-e5a7-4168-a2f0-7471520e3f00,0.00084,Freestanding Glass Makeup Mirror,homedecor,bedroom,Do your makeup properly! Don't make a big mess...,99.00
9,b1a54da3-32d9-4447-9eeb-6c0fcfb75ca0,0.00082,Kettle,housewares,kitchen,Unparalleled for every kitchen,72.99


### Base 솔루션 vs CVR 가중치 솔루션 추천 : user_id = 100

In [8]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn1,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------") 
print("Base Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름

-------------------
user_id:  100
-------------------
Base Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,387f55e6-0eb9-47d6-9ef1-56a80b884430,0.00006,Electric Bass,instruments,strings,This electric bass will delight the most deman...,57.99
1,e09d3267-1a87-4518-9fa7-fa9ed847c80d,0.00059,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
2,b30f5517-9cc2-45ef-87d8-5bb60a443c36,0.00098,Video Camera,electronics,camera,This video camera is perfect for capturing tho...,752.99
3,66725b05-5d69-48f5-bafc-883957a493d1,0.00017,French Horn,instruments,wind,This french horn is perfect for amateur and pr...,192.99
4,6ca2a410-aba9-4c73-9dae-d188412e0e50,0.00005,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
5,a051511e-d3ac-46d3-ac9c-e82fa8e60a3e,0.00078,Video Camera,electronics,camera,This video camera is perfect for capturing tho...,825.99
6,cab4ea76-47e6-4ffe-80dc-ab14f3613547,0.00035,Faultless Speakers,electronics,speaker,Faultless speakers to fill the house with beau...,182.99
7,76fa669b-1611-4f31-8377-55c3e701ced4,0.00133,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
8,3d3f666f-013d-4bed-b694-f5b0623e8335,0.00088,Faultless Headphones,electronics,headphones,This faultless pair of headphones is unequaled...,146.99
9,e2397f59-12e5-454a-99c8-99cd5fe86e44,0.00016,Video Camera,electronics,camera,This video camera is perfect for capturing tho...,805.99


In [9]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn2,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------") 
print("CVR Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름

-------------------
user_id:  100
-------------------
CVR Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,e09d3267-1a87-4518-9fa7-fa9ed847c80d,0.00059,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
1,387f55e6-0eb9-47d6-9ef1-56a80b884430,0.00006,Electric Bass,instruments,strings,This electric bass will delight the most deman...,57.99
2,10a400a4-3afa-4b7b-989e-dc7ed6298efd,0.00074,Drum Set,instruments,percussion,This drum set will delight the most demanding ...,364.99
3,9d1399f1-ec64-44a7-b088-56998f992414,0.00050,Acoustic Guitar,instruments,strings,This acoustic guitar will delight the most dem...,233.99
4,a54fbc82-c85d-4d21-b907-e8cdbbff69d2,0.00020,Acoustic Bass,instruments,strings,This acoustic bass will delight the most deman...,221.99
5,76fa669b-1611-4f31-8377-55c3e701ced4,0.00133,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
6,6ca2a410-aba9-4c73-9dae-d188412e0e50,0.00005,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
7,66725b05-5d69-48f5-bafc-883957a493d1,0.00017,French Horn,instruments,wind,This french horn is perfect for amateur and pr...,192.99
8,c355dd01-24a6-4a99-b70e-539e39f18014,0.00085,Acoustic Guitar,instruments,strings,This acoustic guitar is perfect for amateur an...,172.99
9,e3ba411e-c199-4933-a31f-5eb4260baa4a,0.00057,Italian Cuisine,books,cooking,Learn to cook Italian cuisine with these easy ...,25.99
